In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import os
import logging
import re
import datetime

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
else:
    print("No compatible GPUs found")

## Common constants

In [ ]:
IMG_W = 32
IMG_H = 32
IMG_C = 1

last_epoch = 0
batch_size = 128
latent_dim = 128
SAMPLES_PATH = "samples_AE3"
CHECKPOINT_PATH = "saved_model_AE3"
LOG_DIR = "/qarr/studia/magister/models/gan_logs_AE" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
try:
    checkpoints = os.listdir(CHECKPOINT_PATH)
    last_epoch = np.max([int(re.search(r"-[0-9]+\.", i)[0][1:-1]) for i in checkpoints if
                             re.search(r"-[0-9]+\.", i)])
    print("Detected {} epoch as last checkpoint".format(last_epoch))
except (ValueError,  FileNotFoundError):
    last_epoch = 0
    print("Did not detect any checkpoints to continue from")

## Loading the dataset

In [ ]:
learning_set = np.fromfile('fashion_mnist/train-images-idx3-ubyte', dtype='uint8')
learning_lab = np.fromfile('fashion_mnist/train-labels-idx1-ubyte', dtype='uint8')
test_set = np.fromfile('fashion_mnist/t10k-images-idx3-ubyte', dtype='uint8')
test_lab = np.fromfile('fashion_mnist/t10k-labels-idx1-ubyte', dtype='uint8')

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

In [ ]:
train_examples = 60000
test_examples = 10000
batch_size = 128

resized_train = np.zeros((train_examples, 32, 32))
for i in range(train_examples):
    resized_train[i] = cv2.resize(np.reshape(learning_set[16+28*28*i:16+28*28*(i+1)], (28,28,1)), (32, 32))
resized_train = np.resize(resized_train, (60000, 32, 32, 1))

train_dataset = tf.data.Dataset.from_tensor_slices(resized_train)
def normalize(images):
  images = tf.cast(images, tf.float32)
  images -= 128.0
  images /= 128
  return images

train_dataset =  train_dataset.map(normalize)
train_dataset = train_dataset.cache().repeat().shuffle(train_examples).batch(batch_size)

In [ ]:
plt.imshow(train_dataset.as_numpy_iterator().next()[0], cmap=plt.cm.binary)

In [ ]:
plt.hist(np.ravel(train_dataset.as_numpy_iterator().next()[0]))

### Common helper functions and clearing session

In [ ]:
from gan_arch import *
from commons import *
weight_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

In [ ]:
tf.keras.backend.clear_session()

checkpoint_path_g = f"{CHECKPOINT_PATH}/gencp-{{epoch:02d}}.ckpt"
checkpoint_path_d = f"{CHECKPOINT_PATH}/discp-{{epoch:02d}}.ckpt"
checkpoint_path_e = f"{CHECKPOINT_PATH}/encod-{{epoch:02d}}.ckpt"

In [ ]:
tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = LOG_DIR, histogram_freq=1)

## Defining the model

In [ ]:
d_model = build_discriminator_ref(weight_init, IMG_H=IMG_H, IMG_W=IMG_W, IMG_C=IMG_C)
g_model = build_generator_ref(latent_dim, weight_init, IMG_H=IMG_H, IMG_W=IMG_W, IMG_C=IMG_C)
e_model = build_encoder_ref(latent_dim, weight_init, IMG_H=IMG_H, IMG_W=IMG_W, IMG_C=IMG_C)

### Loading the model from last checkpoint, if able

In [ ]:
if last_epoch:
    d_model.load_weights(checkpoint_path_d.format(epoch=last_epoch))
    g_model.load_weights(checkpoint_path_g.format(epoch=last_epoch))
    e_model.load_weights(checkpoint_path_e.format(epoch=last_epoch))
    print(f"Loading models from epoch {last_epoch}")
else:
    print("No checkpoint to load from")

In [ ]:
train_for = 10 # epochs to train

In [ ]:
gan = GAN_autoencoder(d_model, g_model, e_model, latent_dim)
bce_loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.1)
bce_loss_nl_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.1)
mse_loss = tf.keras.losses.MeanSquaredError()
d_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
g_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
e_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
gan.compile(d_optimizer, g_optimizer, e_optimizer, bce_loss_fn, mse_loss)

In [ ]:
n_samples = 25
noise = np.random.normal(size=(n_samples, latent_dim))

logging.getLogger('tensorflow').setLevel(logging.ERROR)
try:
    for epoch in range(last_epoch, last_epoch+train_for):
        gan.fit(train_dataset, 
                initial_epoch=epoch, 
                epochs=epoch+1, 
                batch_size=batch_size, 
                steps_per_epoch=train_examples//batch_size,
                callbacks=[tboard_callback]
               )
        g_model.save_weights(checkpoint_path_g.format(epoch=epoch+1))
        d_model.save_weights(checkpoint_path_d.format(epoch=epoch+1))
        e_model.save_weights(checkpoint_path_e.format(epoch=epoch+1))

        n_samples = 25
        new_noise = np.random.normal(size=(n_samples, latent_dim))
        new_noise[0:5] = noise[0:5]
        examples = g_model.predict(new_noise)
        save_plot(examples, SAMPLES_PATH, epoch, int(np.sqrt(n_samples)), (IMG_W, IMG_H, IMG_C), plotName="generated_plot_epoch")
except KeyboardInterrupt:
    print("Interrupted")
    
last_epoch += train_for

logging.getLogger('tensorflow').setLevel(logging.WARNING)

## Testing the autoencoder

In [ ]:
n_samples = 25
new_noise = np.random.normal(size=(n_samples, latent_dim))
examples = g_model.predict(new_noise)
canvas = save_plot(examples, SAMPLES_PATH, 0, int(np.sqrt(n_samples)), (IMG_W, IMG_H, IMG_C))

In [ ]:
examples2 = list(train_dataset.take(1).as_numpy_iterator())[0]

In [ ]:
canvas2 = save_plot(examples2, SAMPLES_PATH, 0, int(np.sqrt(n_samples)), (IMG_W, IMG_H, IMG_C))

In [ ]:
examples_noise = e_model.predict(examples)
examples_real_noise = e_model.predict(examples2)
auto_examples = g_model.predict(examples_noise)
auto_examples2 = g_model.predict(examples_real_noise)

In [ ]:
print(np.max(examples2), np.min(examples2))

In [ ]:
canvas_auto = save_plot(auto_examples, SAMPLES_PATH, 0, int(np.sqrt(n_samples)), (IMG_W, IMG_H, IMG_C))

In [ ]:

canvas_auto_real = save_plot(auto_examples2, SAMPLES_PATH, 0, int(np.sqrt(n_samples)), (IMG_W, IMG_H, IMG_C))

In [ ]:
fig, subs = plt.subplots(2,2, figsize=(14,14))
#fig.figure(figsize=(10,10))
subs[0][0].imshow(canvas, cmap = plt.cm.binary)
subs[0][0].set_title("Generowane")
subs[0][1].imshow(canvas2, cmap = plt.cm.binary)
subs[0][1].set_title("Rzeczywiste")
subs[1][0].imshow(canvas_auto, cmap = plt.cm.binary)
subs[1][0].set_title("Generowane -> Autokoder -> Generator")
subs[1][1].imshow(canvas_auto_real, cmap = plt.cm.binary)
subs[1][1].set_title("Rzeczywiste -> Autokoder -> Generator")

 # Training the encoder only

In [ ]:
# noise dataset
@tf.function
def noise_gen(batch_size):
    yield tf.random.normal((batch_size, latent_dim), 0.0, 1.0)

noise_dataset = tf.data.Dataset.from_generator(noise_gen,
                                             args=[batch_size],
                                             output_types=(tf.float32),
                                             output_shapes=((batch_size, latent_dim))
                                            ).prefetch(tf.data.experimental.AUTOTUNE)

class AE_generator(tf.keras.models.Model):
    def __init__(self, generator, encoder):
        super(AE_generator, self).__init__()
        self.generator = generator
        self.encoder = encoder

    def compile(self, optimizer, loss_fn):
        super(AE_generator, self).compile()
        self.optimizer = optimizer
        self.loss_fn = loss_fn
    
    @tf.function
    def train_step(self, noise_input):
        batch_size = tf.shape(real_images)[0]
        with tf.GradientTape() as etape:
            gen_samples = self.generator(noise_input)
            prediction = self.encoder(gen_samples)
            e_loss = loss_fn(random_latent_vectors, encoding_prediction)
        grads = etape.gradient(e_loss, self.encoder.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.encoder.trainable_weights))